# 🚀 YOLOv8 Detection API Server

This notebook runs a Flask API server on Google Colab and exposes it via ngrok.

**Steps:**
1. Upload your `best.pt` model file
2. Run all cells
3. Copy the ngrok URL (looks like: https://xxxx-xx-xx-xx.ngrok.io)
4. Use this URL in your web app to connect to the API

---

In [ ]:
# Install required packages
!pip install -q flask flask-cors ultralytics pyngrok

In [ ]:
# Upload your trained model
from google.colab import files
import os

print("📤 Upload your best.pt model file:")
uploaded = files.upload()

# Rename to banking_model.pt
for filename in uploaded.keys():
    if filename.endswith('.pt'):
        os.rename(filename, 'banking_model.pt')
        print(f"✅ Model saved as banking_model.pt")
        break

In [ ]:
# Create the Flask API server
api_code = '''
from flask import Flask, request, jsonify
from flask_cors import CORS
from ultralytics import YOLO
from PIL import Image
import io
import base64
import time

app = Flask(__name__)
CORS(app)

print("Loading YOLOv8 model...")
model = YOLO('banking_model.pt')
print(f"✅ Model loaded! Classes: {list(model.names.values())}")

@app.route('/health', methods=['GET'])
def health():
    return jsonify({
        'status': 'healthy',
        'model': 'YOLOv8 Banking Detection',
        'classes': model.names
    })

@app.route('/detect', methods=['POST'])
def detect():
    try:
        start_time = time.time()
        
        data = request.json
        if not data or 'image' not in data:
            return jsonify({'error': 'No image provided'}), 400
        
        # Decode base64 image
        image_data = data['image']
        if 'base64,' in image_data:
            image_data = image_data.split('base64,')[1]
        
        image_bytes = base64.b64decode(image_data)
        image = Image.open(io.BytesIO(image_bytes))
        
        # Get confidence threshold (default 0.5)
        conf_threshold = data.get('confidence', 0.5)
        
        # Run detection
        results = model(image, conf=conf_threshold, verbose=False)
        
        # Format detections
        detections = []
        for result in results:
            boxes = result.boxes
            for box in boxes:
                x1, y1, x2, y2 = box.xyxy[0].tolist()
                detections.append({
                    'class': model.names[int(box.cls[0])],
                    'confidence': float(box.conf[0]),
                    'bbox': {
                        'x': x1,
                        'y': y1,
                        'width': x2 - x1,
                        'height': y2 - y1
                    }
                })
        
        processing_time = time.time() - start_time
        
        return jsonify({
            'detections': detections,
            'count': len(detections),
            'processing_time': round(processing_time, 3)
        })
        
    except Exception as e:
        return jsonify({'error': str(e)}), 500

if __name__ == '__main__':
    app.run(port=5000)
'''

with open('api_server.py', 'w') as f:
    f.write(api_code)

print("✅ API server code created")

In [ ]:
# Setup ngrok (you'll need an ngrok account - free)
# Get your authtoken from: https://dashboard.ngrok.com/get-started/your-authtoken

from pyngrok import ngrok, conf
import getpass

print("🔐 Get your ngrok authtoken from: https://dashboard.ngrok.com/get-started/your-authtoken")
print("(Sign up for free if you don't have an account)\n")

authtoken = getpass.getpass("Enter your ngrok authtoken: ")
ngrok.set_auth_token(authtoken)

print("✅ Ngrok configured!")

In [ ]:
# Start the API server with ngrok tunnel
import threading
import time
from pyngrok import ngrok

# Start Flask in background thread
def run_flask():
    import subprocess
    subprocess.run(['python', 'api_server.py'])

flask_thread = threading.Thread(target=run_flask, daemon=True)
flask_thread.start()

# Wait for Flask to start
time.sleep(3)

# Create ngrok tunnel
public_url = ngrok.connect(5000, bind_tls=True)

print("\n" + "="*60)
print("🚀 YOLOv8 Detection API is LIVE!")
print("="*60)
print(f"\n📡 Public URL: {public_url}")
print(f"\n🔗 API Endpoints:")
print(f"   Health Check: {public_url}/health")
print(f"   Detection:    {public_url}/detect")
print("\n💡 Copy the URL above and use it in your web app!")
print("\n⚠️  Keep this notebook running - closing it will stop the API")
print("="*60)

# Keep running
try:
    while True:
        time.sleep(1)
except KeyboardInterrupt:
    print("\n🛑 Server stopped")

---

## 📝 Using the API in Your Web App

Once you have the ngrok URL, update your web app to call this API:

```javascript
// Example API call
const apiUrl = 'https://xxxx-xx-xx-xx.ngrok.io'; // Your ngrok URL

async function detectObjects(imageElement) {
    const canvas = document.createElement('canvas');
    canvas.width = imageElement.width;
    canvas.height = imageElement.height;
    canvas.getContext('2d').drawImage(imageElement, 0, 0);
    const base64Image = canvas.toDataURL('image/jpeg');
    
    const response = await fetch(`${apiUrl}/detect`, {
        method: 'POST',
        headers: { 'Content-Type': 'application/json' },
        body: JSON.stringify({ 
            image: base64Image,
            confidence: 0.5 
        })
    });
    
    const result = await response.json();
    return result.detections;
}
```

**Response format:**
```json
{
    "detections": [
        {
            "class": "person",
            "confidence": 0.92,
            "bbox": { "x": 100, "y": 50, "width": 200, "height": 400 }
        }
    ],
    "count": 1,
    "processing_time": 0.123
}
```